# Debug helpers

This notebook contains a bunch of scripts that can help debug some issues.

In [ ]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python  # type: ignore

## Delete a Kafka topic

In [ ]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import TopicAlreadyExistsError

#
#  CHANGE THIS TO THE TOPIC YOU WANT TO DELETE
#
topic = 'clicks'

admin_client = KafkaAdminClient(bootstrap_servers="localhost:9092")
admin_client.delete_topics([topic])

## Show the last 10 messages on a Kafka topic

In [ ]:
import json
from datetime import datetime

from kafka import KafkaConsumer, TopicPartition

client = "localhost:9092"
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'],
                         enable_auto_commit=False,
                         auto_offset_reset='latest')

#
#  CHANGE THIS TO THE TOPIC YOU WANT TO DEBUG
#
topic = 'clicks'
# topic = 'clicks-cleaned'


tp = TopicPartition(topic,0)
consumer.assign([tp])

consumer.seek_to_end(tp)
lastOffset = consumer.position(tp)
print(f"Last offset: {lastOffset}")

tailOffset = lastOffset - 10
if tailOffset < 0:
    print("Topic has no messages!")
else :
    consumer.seek(tp, tailOffset)    

    for message in consumer:
        print(f"RAW MESSAGE VALUE: {message.value}")
        try:
            click = json.loads(message.value)
            print(json.dumps(click, indent=4))
        except:
            print("FAILED to decode message")
        print()
    #     ts = click.get('ts_ingest') or 0
    #     ts= datetime.fromtimestamp(ts/1000.0)
    #     print(f"timestamp: {ts_format}")
        if message.offset >= lastOffset - 1:
                break

## Follow a Kafka topic

This script prints the content of messages of a Kafka topic when they get produced.

In [ ]:
import json
from datetime import datetime
from IPython.display import clear_output

from kafka import KafkaConsumer, TopicPartition

client = "localhost:9092"
consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'],
                         enable_auto_commit=False,
                         auto_offset_reset='latest')

#
#  CHANGE THIS TO THE TOPIC YOU WANT TO DEBUG
#
topic = 'clicks-cleaned'

consumer.subscribe(topic)

i = 0

for message in consumer:
    print(f"RAW MESSAGE VALUE: {message.value}")
    try:
        click = json.loads(message.value)
        print(json.dumps(click, indent=4))
    except:
        print("FAILED to decode message")
    print()
#     ts = click.get('ts_ingest') or 0
#     ts= datetime.fromtimestamp(ts/1000.0)
#     print(f"timestamp: {ts_format}")
    if i > 500:
        clear_output()
        i = 0
    i = i+1